In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
try:
    import empyrical as emp
except:
    emp = None
import tushare as ts
import time
import random

from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq
from spider.spider_coinmarketcap import Spider_coinmarketcap
from spider.spider_okex import Spider_okex

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

CONF = Config('../conf/secret.yaml').data[0]
ts_token = CONF['TUSHARE']['TOKEN']
ts.set_token(ts_token)
pro = ts.pro_api()

CONF = Config().data[0]
MONGODB = CONF['MONGODB']
NASDAQ = CONF['NASDAQ']
NASDAQ100 = CONF['NASDAQ100']
CRYPTOCURRENCY = CONF['CRYPTOCURRENCY']
CRYPTOCURRENCY = list(CRYPTOCURRENCY.keys())
CRYPTOCURRENCYSYMBOL = CONF['CRYPTOCURRENCYSYMBOL']

ETF_LIST = {
    '399300', '000905', '000016', '399006', 
    '159934', '518800', '518880', '159937', '512400', '511310', '511290', 
    '511260', '511020', '511270', '159926', '511010', '511280', '511030', 
    '511220', '511880', '512170', '510410', '512300', '510170', '159928', 
    '512600', '510630', '512340', '159929', '512610', '159967', '510660', 
    '159966', '512690', '515000', '513880', '513520', '159944', '512120', 
    '159938', '512480', '512010', '513000', '512390', '513800', '512290', 
    '510150', '159971', '159964', '159956', '510290', '512700', '159958', 
    '510440', '159908', '159957', '159955', '159948', '512760', '159915', 
    '159952', '512890', '512800', '512820', '510550', '159962', '512590', 
    '159935', '512510', '510510', '510560', '510890', '159973', '510120', 
    '512500', '510580', '512270', '510590', '510500', '159922', '510270', 
    '510220', '512670', '510030', '510210', '512310', '512150', '159932', 
    '510130', '512260', '512770', '512860', '159949', '512550', '512750', 
    '512360', '512580', '512380', '510880', '512280', '512850', '512920', 
    '159951', '512910', '512180', '159943', '512090', '512520', '159923', 
    '512990', '159965', '512040', '510180', '159903', '512160', '510360', 
    '510380', '159936', '510800', '159905', '159925', '159961', '510330', 
    '159912', '510300', '510390', '510010', '159919', '510310', '159911', 
    '510600', '510850', '510710', '510050', '512200', '159901', '510110', 
    '512980', '510680', '512330', '510060', '510090', '510020', '510160', 
    '510650', '512100', '512960', '159953', '159918', '159959', '510070', 
    '512950', '510810', '510190', '510820', '512780', '159913', '159907', 
    '159933', '159916', '512680', '512870', '159910', '159902', '512640', 
    '159940', '159931', '510230', '512810', '510430', '512560', '512660', 
    '159968', '159930', '159945', '512220', '159939', '159909', '159920', 
    '513660', '159906', '513600', '513900', '159963', '513680', '159960', 
    '159954', '510900', '512070', '512570', '512900', '512880', '512000', 
    '513100', '513500', '513030', '513050', '159941', '512790', '512650', 
    '512710', '512720', '159969', '512930', '510350',
}

ETF_LIST = [
    '399300', '000905', '000016', '399006', 
    '159920', # 恒生ETF 恒生指数
    '510900', # H股ETF 恒生国企
    '501021', # 香港中小 香港中小
    '513500', # 标普500 标普500
    '513100', # 纳指ETF 纳斯达克100
    '513000', # 225ETF 日经225
    '513030', # 德国30 德国DAX
    '513050', # 中概互联 中证海外中国互联网50
    '501018', # 南方原油 FOF
    '160416', # 石油基金 标普全球石油指数
    '162411', # 华宝油气 标普石油天然气上游股票指数
    '162719', # 广发石油 道琼斯美国石油开发与生产指数
    '159934', # 黄金ETF 沪金99
    '159937', # 博时黄金 沪金99
    '518800', # 黄金基金 沪金99
    '518880', # 黄金ETF 沪金99
    '161226', # 白银基金 上海期货交易所白银期货主力合约
    '511010', # 国债ETF 5年国债
    '511260', # 十年国债 10年国债
    '511660', # 建信添益
    '511690', # 交易货币
    '511700', # 场内货币
    '511810', # 理财金H
    '511850', # 财富宝E
    '511880', # 银华日利
    '511990', # 华宝添益
]

pd

In [ ]:
%%time

IS_DOWNLOAD_DATA = True

spider_n = Spider_nasdaq()
spider_c = Spider_coinmarketcap()
spider_o = Spider_okex()

for symbol in ETF_LIST:
    if not IS_DOWNLOAD_DATA:
        break
        
    print(symbol)

    # 指数
    if symbol in ['399300', '000905', '000016', '399006']:
        df = ts.get_k_data(code=symbol, index=True, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
#         datafile = '../database/market/%s_lite.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        
    # CRYPTOCURRENCY
    elif symbol in CRYPTOCURRENCYSYMBOL:
        datafile = spider_o.get_coin_data(symbol, lite=True)
        print(datafile)
        
    # ETF
    else:
        df = ts.get_k_data(code=symbol, ktype='D', autype='qfq', index=False, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
#         datafile = '../database/market/%s_lite.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        time.sleep(1)

    time.sleep(1)

In [ ]:
%%time

stock_df_dict = {}

for symbol in ETF_LIST:
    stock_data_file = '../database/market/%s.csv' % symbol
#     stock_data_file = '../database/market/%s_lite.csv' % symbol
    if symbol in CRYPTOCURRENCYSYMBOL:
        stock_data_file = '../database/market/%s_OKEX.csv' % symbol
        stock_data_file = '../database/market/%s_OKEX_lite.csv' % symbol
    try:
        stock_df = pd.read_csv(stock_data_file)
    except:
        print('ERROR_READING_CSV', symbol)
        continue

    # 筛选字段
    stock_df = stock_df.loc[:, ['date', 'close', 'volume']]

    # 抛弃空值异常值
    stock_df.dropna(axis=0, how='any', inplace=True)

    # 格式化日期
    # 445 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
    stock_df = stock_df.assign(date=pd.to_datetime(stock_df['date']))  # need .index.to_period('D')

    # 用日期作索引，日期升序排序
    # 95.1 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
    if symbol in NASDAQ100 or symbol in CRYPTOCURRENCYSYMBOL:
        stock_df = stock_df[1:]
        stock_df = stock_df[::-1]
    stock_df.set_index(['date'], inplace=True)
    stock_df.index = stock_df.index.to_period('D')
    
    if len(stock_df) < 750:
        print('NOT ENOUGH DATA', symbol)
        continue
        
    print(symbol, stock_df.iloc[-1]['volume'])

#     # 计算每天涨跌幅
#     N = MONITOR_TARGET[symbol]['N']
#     stock_df['N_sht'] = stock_df.close.shift(N-1)
#     stock_df['N_chg'] = (stock_df.close.shift(0) - stock_df.close.shift(N-1)) / stock_df.close.shift(N-1)

#     for MA_PARAM in MONITOR_TARGET[symbol]['MA']:
#         stock_df['MA_%d' % MA_PARAM] = stock_df['close'].rolling(MA_PARAM).mean()
    
#     for ROLLINGMAX_PARAM in MONITOR_TARGET[symbol]['ROLLINGMAX']:
#         stock_df['ROLLINGMAX_%d' % ROLLINGMAX_PARAM] = stock_df['close'].rolling(ROLLINGMAX_PARAM).max()
        
#     for ROLLINGMIN_PARAM in MONITOR_TARGET[symbol]['ROLLINGMIN']:
#         stock_df['ROLLINGMIN_%d' % ROLLINGMIN_PARAM] = stock_df['close'].rolling(ROLLINGMIN_PARAM).min()
    
    # 减少数据
    stock_df.dropna(how='any', inplace=True)
    
    stock_df_dict[symbol] = stock_df

In [ ]:
# for symbol in ETF_LIST:
#     symbol
#     stock_df_dict[symbol].head(1)
#     stock_df_dict[symbol].tail(1)

In [ ]:
corr_df = stock_df_dict['399300'].copy()

for symbol in stock_df_dict:
    corr_df[symbol] = stock_df_dict[symbol]['close']

corr_df.drop(columns=['close'], inplace=True)
corr_df.dropna(how='any', inplace=True)
# corr_df

res_df = corr_df.corr()
res_df.sort_values(by=['399300', '399006'])
res_df['399300'].sort_values(ascending=True)

In [ ]:
monitor_df = pd.DataFrame(columns=['SYMBOL', 'NAME', 'DATE', 'CLOSE', 'MA', 'MADIFF', 'N_sht', 'N_chg'])
#  'BUY', 'BUYDIFF', 'SELL', 'SELLDIFF'
for symbol in MONITOR_TARGET:
    df = stock_df_dict[symbol].iloc[-30:].copy()
    
    df.reset_index(drop=False, inplace=True)
    # df = df.astype(dtype={'date': 'datetime64[ns]'})
    df['date'] = df['date'].apply(lambda x: x.to_timestamp().to_datetime64())
    df.set_index('date', inplace=True)
    
    today_market = df.iloc[-1]
    now_point = today_market.close
    for col in df.columns:
        if 'ROLLINGMAX' in col:
            buy_point = today_market[col]
            buy_diff = (buy_point - now_point) / now_point * 100
        elif 'ROLLINGMIN' in col:
            sell_point = today_market[col]
            sell_diff = (now_point - sell_point) / now_point * 100
        elif 'MA' in col:
            ma = today_market[col]
            ma_diff = (now_point - ma) / now_point * 100
        elif 'N_chg' in col:
            n_chg = today_market[col] * 100
    
    monitor_df = monitor_df.append({
        'SYMBOL': symbol, 
        'NAME': MONITOR_TARGET[symbol]['NAME'],
        'DATE': today_market.name.date(), 
        'CLOSE': now_point, 
#         'BUY': buy_point, 
#         'BUYDIFF': '+%.2f%%' % buy_diff, 
#         'SELL': sell_point, 
#         'SELLDIFF': '-%.2f%%' % sell_diff,
        'MA': ma,
        'MADIFF': '%.2f%%' % ma_diff,
        'N_sht': today_market.N_sht, 
        'N_chg': '%.2f%%' % n_chg,
    }, ignore_index=True)
    
    title = symbol
    title = '%s, %s, now=%.2f, N_chg=%.2f(%.2f%%), MADIFF=%.2f(%.2f%%)' % \
        (symbol, today_market.name.date(), now_point, today_market.N_sht, n_chg, ma, ma_diff)
    
#     df.drop(columns=['N_chg'], inplace=True)
#     ax = df.plot(kind='line', title=title, linewidth=0.9, grid=True, figsize=(19, 5))
#     ax.yaxis.tick_right()
    
#     display_charts(df, chart_type='stock', kind='line', title=title, figsize=(1000, 600))


monitor_df

In [ ]:
12500 / 12339.6
309 / 308.04
132 / 128.88
4000 / 3170
1.16 ** 12
# (1.18 ** (1 / 30)) ** 365

In [ ]:
# df = ts.get_realtime_quotes(['sh', 'sz', 'hs300', 'sz50', 'sh000905', 'zxb', 'cyb'])
# df = ts.get_realtime_quotes(['hs300', 'sh000905', 'cyb'])
# df = ts.get_realtime_quotes(list(MONITOR_TARGET.keys()))
# df
# list(MONITOR_TARGET.keys())

# ts
# pro
df = pro.index_daily(ts_code='399300.SZ', start_date='20190601', end_date='20190611')
df

df = ts.get_k_data(code='399300', index=True, start='2019-06-01', end=time.strftime('%Y-%m-%d'))
df